# 'Group Project'

### Introduction

* provide some relevant background information on the topic so that someone unfamiliar with it will be prepared to understand the rest of your report (e.g. section 5.4.2)
* clearly state the question you tried to answer with your project
* identify and fully describe the dataset that was used to answer the question

Our model uses KNN classification to label players as "committed" or "not interested" based on their total playtime in minutes and the number of sessions they have played. Committed players are those that can be expected to contribute more longterm.

In [1]:
# Import packages
import pandas as pd
import altair as alt
import numpy as np
from datetime import timedelta
from sklearn import set_config
from sklearn.preprocessing import StandardScaler
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import make_pipeline                           
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score, precision_score
from sklearn.model_selection import GridSearchCV

np.random.seed(1)

# Simplify working with large datasets in Altair J AI FAIT CA MAIS CA NE FONCTIONNE PAS POUR LE df DONC A REVOIR
#alt.data_transformers.enable('vegafusion')

# Output dataframes instead of arrays
set_config(transform_output="pandas")

### Preprocessing of the data

In [2]:
# Load the data
players = pd.read_csv('players.csv', usecols=["experience", "subscribe", "hashedEmail", "played_hours", "age"])
sessions = pd.read_csv('sessions.csv', parse_dates=['start_time','end_time'], dayfirst=True, usecols=["hashedEmail", "start_time", "end_time"])

In [20]:
# Order the experience from 1 to 5, 5 being the greatest

players["experience"] = players["experience"].replace({
    "Beginner" : "1-Beginner",
    "Amateur" : "2-Amateur",
    "Regular" : "3-Regular",
    "Pro" : "4-Pro",
    "Veteran" : "5-Veteran"
})

# Merge dataframes, calculate playtime for each session, convert times to PST (UTC assumed)

tidywhole = (
    pd.merge(sessions, players, how="inner", on=["hashedEmail"])
    .assign(played_minutes = (sessions["end_time"]-sessions["start_time"]) / timedelta(minutes=1))
)
tidywhole["start_time"] = tidywhole["start_time"] + pd.Timedelta(hours=-8)
tidywhole["end_time"]   = tidywhole["end_time"]   + pd.Timedelta(hours=-8)

tidywhole

# out of 196 players some have never played:
#print(len(tidywhole["hashedEmail"].unique()))   #= 125


,hashedEmail,start_time,end_time,experience,subscribe,played_hours,age,played_minutes
0,bfce39c89d6549f2bb94d8064d3ce69dc3d7e72b38f431...,2024-06-30 10:12:00,2024-06-30 10:24:00,3-Regular,True,223.1,17,12.0
1,36d9cbb4c6bc0c1a6911436d2da0d09ec625e43e6552f5...,2024-06-17 15:33:00,2024-06-17 15:46:00,2-Amateur,True,53.9,17,13.0
2,f8f5477f5a2e53616ae37421b1c660b971192bd8ff77e3...,2024-07-25 09:34:00,2024-07-25 09:57:00,2-Amateur,True,150.0,16,23.0
3,bfce39c89d6549f2bb94d8064d3ce69dc3d7e72b38f431...,2024-07-24 19:22:00,2024-07-24 19:58:00,3-Regular,True,223.1,17,36.0
4,36d9cbb4c6bc0c1a6911436d2da0d09ec625e43e6552f5...,2024-05-25 08:01:00,2024-05-25 08:12:00,2-Amateur,True,53.9,17,11.0
...,...,...,...,...,...,...,...,...
1530,36d9cbb4c6bc0c1a6911436d2da0d09ec625e43e6552f5...,2024-05-10 15:01:00,2024-05-10 15:07:00,2-Amateur,True,53.9,17,6.0
1531,7a4686586d290c67179275c7c3dfb4ea02f4d317d9ee0e...,2024-06-30 20:08:00,2024-06-30 20:19:00,5-Veteran,True,1.6,23,11.0
1532,fd6563a4e0f6f4273580e5fedbd8dda64990447aea5a33...,2024-07-28 07:36:00,2024-07-28 07:57:00,2-Amateur,True,56.1,23,21.0
1533,fd6563a4e0f6f4273580e5fedbd8dda64990447aea5a33...,2024-07-24 22:15:00,2024-07-24 22:22:00,2-Amateur,True,56.1,23,7.0


- *explain that we want to predict a category, hence classifier not regression*
- *the data do not have a column to identify committed players from non-committed players*
- */!\ need to determine metrics for committed player (=category) to feed the prediction model*
- *several metrics available*

 - For the following coding, we will need to use KNN classification, as our goal is to predict a categorical rather than a numerical variable.
 - First things first we need to determine the players that are classified committed and those who are not interested (I said those who played 2weeks or more are committed)

In [4]:
# Calculate the period over which each person has played
# under the assumption that the longer the period the more the player is committed (example: 2months vs 2days)
grouped = tidywhole.groupby(['hashedEmail'])
period = grouped["end_time"].max() - grouped["start_time"].min()
period = period.dt.total_seconds() / (24 * 3600)   #converted to number of days (decimal)
period_df= period.reset_index()
period_df.columns = ["hashedEmail","period (days)"]

commit = alt.Chart(period_df.assign(playerID=period_df.index+1)).mark_circle().encode(
    x=alt.X("playerID").title("Player"),
    y=alt.Y("period (days)").title("Period over which has played (in days)"),
)

commit_zoom = alt.Chart(period_df.assign(playerID=period_df.index+1)).mark_circle(clip=True).encode(
    x=alt.X("playerID").title("Player"),
    y=alt.Y("period (days)").scale(domain=[0, 28]).title("Period over which has played (in days)"),
)

hline = alt.Chart(period_df).mark_rule(strokeDash=[10], size=1, color='darkorange').encode( y=alt.datum(14))

commit+hline | commit_zoom+hline


alt.HConcatChart(...)

In [5]:
# so few players have played for longer than 2 weeks = 25
more_than_2wks = period[period>14]
# or even 1 week = 28
more_than_1wk = period[period>7]

# Let's consider players that have played the game for over 2 weeks as committed players
commit_list = list(more_than_2wks.index)  # this is their list (emails)


In [6]:
# Calculate total playtime for each player

playtime = grouped['played_minutes'].sum()
playtime_df = playtime.reset_index()
playtime_df.columns = ['hashedEmail', 'total playtime (min)']

playtime_df

,hashedEmail,total playtime (min)
0,0088b5e134c3f0498a18c7ea6b8d77b4b0ff1636fc9335...,106.0
1,060aca80f8cfbf1c91553a72f4d5ec8034764b05ab59fe...,30.0
2,0ce7bfa910d47fc91f21a7b3acd8f33bde6db57912ce02...,11.0
3,0d4d71be33e2bc7266ee4983002bd930f69d304288a866...,418.0
4,0d70dd9cac34d646c810b1846fe6a85b9e288a76f5dcab...,70.0
...,...,...
120,fc0224c81384770e93ca717f32713960144bf0b52ff676...,16.0
121,fcab03c6d3079521e7f9665caed0f31fe3dae6b5ccb86e...,80.0
122,fd6563a4e0f6f4273580e5fedbd8dda64990447aea5a33...,4795.0
123,fe218a05c6c3fc6326f4f151e8cb75a2a9fa29e22b110d...,9.0


In [7]:
# Calculate average session duration for each player

avg_duration = grouped['played_minutes'].mean()
avg_duration_df = avg_duration.reset_index()
avg_duration_df.columns = ['hashedEmail', 'average session duration (min)']

avg_duration_df

,hashedEmail,average session duration (min)
0,0088b5e134c3f0498a18c7ea6b8d77b4b0ff1636fc9335...,53.000000
1,060aca80f8cfbf1c91553a72f4d5ec8034764b05ab59fe...,30.000000
2,0ce7bfa910d47fc91f21a7b3acd8f33bde6db57912ce02...,11.000000
3,0d4d71be33e2bc7266ee4983002bd930f69d304288a866...,32.153846
4,0d70dd9cac34d646c810b1846fe6a85b9e288a76f5dcab...,35.000000
...,...,...
120,fc0224c81384770e93ca717f32713960144bf0b52ff676...,16.000000
121,fcab03c6d3079521e7f9665caed0f31fe3dae6b5ccb86e...,80.000000
122,fd6563a4e0f6f4273580e5fedbd8dda64990447aea5a33...,15.467742
123,fe218a05c6c3fc6326f4f151e8cb75a2a9fa29e22b110d...,9.000000


In [8]:
# Calculate return rate
# defined as the number of sessions per player normalised by period the player has played

nb_sessions = grouped.size()
return_rate = nb_sessions / period       # nb of sessions per day 
weekly_return_rate = return_rate * 7     # nb of sessions per week 
# not sure how to interpret these
# numbers are correct but someone who played only for 15min, once, will have a rate of 1/(15/60/24)*7 = 672 = larger number than a recurring player

# Alternative definition
# defined as the total playtime per player normalised by average duration of a session across all players

avg_duration_all = tidywhole['played_minutes'].mean()
return_rate2 = playtime / avg_duration_all
return_rate2_df = return_rate2.reset_index()
return_rate2_df.columns = ['hashedEmail', 'return rate']
return_rate2_df


,hashedEmail,return rate
0,0088b5e134c3f0498a18c7ea6b8d77b4b0ff1636fc9335...,2.084216
1,060aca80f8cfbf1c91553a72f4d5ec8034764b05ab59fe...,0.589873
2,0ce7bfa910d47fc91f21a7b3acd8f33bde6db57912ce02...,0.216287
3,0d4d71be33e2bc7266ee4983002bd930f69d304288a866...,8.218890
4,0d70dd9cac34d646c810b1846fe6a85b9e288a76f5dcab...,1.376369
...,...,...
120,fc0224c81384770e93ca717f32713960144bf0b52ff676...,0.314599
121,fcab03c6d3079521e7f9665caed0f31fe3dae6b5ccb86e...,1.572993
122,fd6563a4e0f6f4273580e5fedbd8dda64990447aea5a33...,94.281289
123,fe218a05c6c3fc6326f4f151e8cb75a2a9fa29e22b110d...,0.176962


In [9]:
# Concatenate all these data in a tidy dataframe

nb_sessions_df = nb_sessions.reset_index()
nb_sessions_df.columns = ['hashedEmail', 'nb of sessions']

df = (
    playtime_df
    .merge(avg_duration_df, on='hashedEmail')
    .merge(return_rate2_df, on='hashedEmail')
    .merge(nb_sessions_df, on='hashedEmail')
    .merge(period_df, on='hashedEmail')
)

# Add a column for the commitment identifier

df = df.assign(identifier=np.where(df['hashedEmail'].isin(commit_list), 'committed', 'not interested'))
df


,hashedEmail,total playtime (min),average session duration (min),return rate,nb of sessions,period (days),identifier
0,0088b5e134c3f0498a18c7ea6b8d77b4b0ff1636fc9335...,106.0,53.000000,2.084216,2,0.079861,not interested
1,060aca80f8cfbf1c91553a72f4d5ec8034764b05ab59fe...,30.0,30.000000,0.589873,1,0.020833,not interested
2,0ce7bfa910d47fc91f21a7b3acd8f33bde6db57912ce02...,11.0,11.000000,0.216287,1,0.007639,not interested
3,0d4d71be33e2bc7266ee4983002bd930f69d304288a866...,418.0,32.153846,8.218890,13,8.829861,not interested
4,0d70dd9cac34d646c810b1846fe6a85b9e288a76f5dcab...,70.0,35.000000,1.376369,2,1.150694,not interested
...,...,...,...,...,...,...,...
120,fc0224c81384770e93ca717f32713960144bf0b52ff676...,16.0,16.000000,0.314599,1,0.011111,not interested
121,fcab03c6d3079521e7f9665caed0f31fe3dae6b5ccb86e...,80.0,80.000000,1.572993,1,0.055556,not interested
122,fd6563a4e0f6f4273580e5fedbd8dda64990447aea5a33...,4795.0,15.467742,94.281289,310,160.952083,committed
123,fe218a05c6c3fc6326f4f151e8cb75a2a9fa29e22b110d...,9.0,9.000000,0.176962,1,0.006250,not interested


### Visualization of the data

In [10]:
# Draw scatter plots that will help us find the relevant variables for the predictive model
# available variables are total playtime (min), average session duration (min), return rate and nb of sessions

# 1st step is to standardize the variables

preprocessor = make_column_transformer(
    (StandardScaler(), make_column_selector(dtype_include="number")),
    remainder="passthrough",
)
#preprocessor

preprocessor.fit(df)
scaled_df = preprocessor.transform(df)
scaled_df


,standardscaler__total playtime (min),standardscaler__average session duration (min),standardscaler__return rate,standardscaler__nb of sessions,standardscaler__period (days),remainder__hashedEmail,remainder__identifier
0,-0.223530,0.633796,-0.223530,-0.249749,-0.429325,0088b5e134c3f0498a18c7ea6b8d77b4b0ff1636fc9335...,not interested
1,-0.256343,-0.077687,-0.256343,-0.274044,-0.430732,060aca80f8cfbf1c91553a72f4d5ec8034764b05ab59fe...,not interested
2,-0.264546,-0.665434,-0.264546,-0.274044,-0.431047,0ce7bfa910d47fc91f21a7b3acd8f33bde6db57912ce02...,not interested
3,-0.088823,-0.011060,-0.088823,0.017492,-0.220679,0d4d71be33e2bc7266ee4983002bd930f69d304288a866...,not interested
4,-0.239073,0.076983,-0.239073,-0.249749,-0.403790,0d70dd9cac34d646c810b1846fe6a85b9e288a76f5dcab...,not interested
...,...,...,...,...,...,...,...
120,-0.262388,-0.510764,-0.262388,-0.274044,-0.430964,fc0224c81384770e93ca717f32713960144bf0b52ff676...,not interested
121,-0.234755,1.469016,-0.234755,-0.274044,-0.429904,fcab03c6d3079521e7f9665caed0f31fe3dae6b5ccb86e...,not interested
122,1.800954,-0.527229,1.800954,7.233014,3.406712,fd6563a4e0f6f4273580e5fedbd8dda64990447aea5a33...,committed
123,-0.265410,-0.727302,-0.265410,-0.274044,-0.431080,fe218a05c6c3fc6326f4f151e8cb75a2a9fa29e22b110d...,not interested


In [11]:
# 2nd step is to draw a colored scatter plot to visualize the relationship between the variables 

pltime_duration = alt.Chart(scaled_df).mark_circle().encode(
    x=alt.X('standardscaler__total playtime (min)').title("total playtime (standardized)"),
    y=alt.Y('standardscaler__average session duration (min)').title("average session duration (standardized)"),
    color=alt.Color('remainder__identifier').title("Commitment").scale(scheme="set2")
)

pltime_number = alt.Chart(scaled_df).mark_circle().encode(
    x=alt.X('standardscaler__total playtime (min)').title("total playtime (standardized)"),
    y=alt.Y('standardscaler__nb of sessions').title("number of sessions (standardized)"),
    color=alt.Color('remainder__identifier').title("Commitment").scale(scheme="set2")
)

pltime_number2 = alt.Chart(df).mark_circle().encode(
    x=alt.X('total playtime (min)').scale(type='log').title("total playtime (min)"),
    y=alt.Y('nb of sessions').scale(type='log').title("number of sessions"),
    color=alt.Color('identifier').title("Commitment").scale(scheme="set2")
)

pltime_duration | pltime_number | pltime_number2

# it turns out that average session duration is not a useful variable for the predictive model since players who are not committed have played for a wide range of durations
# the total playtime and number of sessions (or return rate) appear to be more useful variables since they highlight separate clusters for committed and not committed players
# also shown with non standardised variables on a log scale (far right)


alt.HConcatChart(...)

In [12]:
scaled_df['remainder__identifier'].value_counts(normalize=True)
# there are 20% of committed players (=25) in the whole dataset


remainder__identifier
not interested    0.8
committed         0.2
Name: proportion, dtype: float64

In [13]:
# Balance the dataset ??
# as it is, the proportion of committed players is much lower than that of non committed players
# this means that, with high values of K, the model is more likely to predict 'non committed' than 'committed'

committed_player = df[df['identifier'] == 'committed']
noncommitted_player = df[df['identifier'] == 'not interested']
committed_player_upsample = committed_player.sample(
    n=noncommitted_player.shape[0], replace=True
)
upsampled_df = pd.concat((committed_player_upsample, noncommitted_player))
upsampled_df['identifier'].value_counts(normalize=True)

identifier
committed         0.5
not interested    0.5
Name: proportion, dtype: float64

### K-nearest neighbors classification

In [14]:
# Follow the workflow for performing K-nearest neighbors classification
# 1. Split the dataset into training and test sets
# 2. Choose the best K parameter with 5-fold cross-validation
# 3. Predict and evaluate performance on the test set


In [15]:
# 1. Split the dataset into training and test sets

# Let python split the data into a training set and a test set to not bias the predictive model
# We use 65% of the data for training, and 35% for testing
                           
commit_train, commit_test = train_test_split(df, train_size=0.65, stratify=df['identifier'])

commit_train['identifier'].value_counts(normalize=True)   # proportion of committed vs not interested is preserved in the training set


identifier
not interested    0.802469
committed         0.197531
Name: proportion, dtype: float64

In [16]:
# 2. Choose the best K parameter with 5-fold cross-validation

# Create the K-NN model with K left unspecified
knn_tune = KNeighborsClassifier()

# Create the centering / scaling preprocessor
commit_preprocessor = make_column_transformer(
#   (StandardScaler(), ['total playtime (min)', 'nb of sessions']),
    (StandardScaler(), make_column_selector(dtype_include="number"))
)

tune_pipeline = make_pipeline(commit_preprocessor, knn_tune)

# Specify the range of K values to try
parameter_grid = {"kneighborsclassifier__n_neighbors": [1,3,5,7,9,11,15,24,31]}   # no need to try large values given the small dataset

# Create the GridSearchCV object with 5-fold cross-validation
commit_tune_grid = GridSearchCV(
    estimator=tune_pipeline,
    param_grid=parameter_grid,
    cv=5
)

# Run to find the best K parameter
X = commit_train[['total playtime (min)', 'nb of sessions']]
Y = commit_train['identifier']

commit_tune_grid.fit(X,Y)
accuracies_grid = pd.DataFrame(commit_tune_grid.cv_results_)[["param_kneighborsclassifier__n_neighbors","mean_test_score","std_test_score"]]
accuracies_grid["Standard error"] = accuracies_grid["std_test_score"] / 5**(1/2)
accuracies_grid.columns=["Neighbors","Accuracy","Standard deviation","Standard error"]

accuracies_grid


,Neighbors,Accuracy,Standard deviation,Standard error
0,1,0.938971,0.054312,0.024289
1,3,0.951471,0.044447,0.019877
2,5,0.938971,0.037247,0.016657
3,7,0.938971,0.037247,0.016657
4,9,0.913971,0.028911,0.012929
5,11,0.913971,0.028911,0.012929
6,15,0.877206,0.053561,0.023953
7,24,0.840441,0.044943,0.020099
8,31,0.802941,0.019118,0.008550


In [17]:
# accuracy best for K=3
commit_tune_grid.best_params_

# Lets compare this to the performance of the majority classifier (approximately the proportion of the majority class in the training set = 80%)
# with an accuracy of 95.1% the predictive model seems to do better
# note that using 31 neighbors gets the performance close to that of the majority classifier


{'kneighborsclassifier__n_neighbors': 3}

In [18]:
# 3. Predict and evaluate performance on the test set

commit_test["predicted"] = commit_tune_grid.predict(commit_test[['total playtime (min)', 'nb of sessions']])
commit_test[['identifier', "predicted"]]

# Evaluate model performance
accuracy = commit_tune_grid.score(
              commit_test[['total playtime (min)', 'nb of sessions']],
              commit_test['identifier']
           )
print(accuracy)

pd.crosstab(
    commit_test['identifier'],
    commit_test["predicted"]
)


0.9545454545454546


predicted,committed,not interested
identifier,,
committed,7,2
not interested,0,35


In [19]:
# Exploratory plots to check whether committed players have a routine

tidywhole['identifier'] = np.where(tidywhole['hashedEmail'].isin(commit_list), 'committed', 'not interested')
tidywhole

domain = ['committed', 'not interested']
range_ = ['lightblue', 'red']

scat = alt.Chart(tidywhole.reset_index()).mark_circle().encode(
    x=alt.X("index").title('Play session'),
    y=alt.Y("start_time:T", timeUnit='hoursminutes').title('Time when played'),
    color=alt.Color('identifier').title("Commitment").scale(domain=domain, range=range_)
)
scat2 = alt.Chart(tidywhole.reset_index()).mark_circle().encode(
    x=alt.X("index").title('Play session'),
    y=alt.Y("start_time:N", timeUnit='day').title('Day when played'),
    color=alt.Color('identifier').title("Commitment").scale(domain=domain, range=range_)
)

scat | scat2
# There is no correlation between commitment and time or day of play

alt.HConcatChart(...)